In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [4]:
df = pd.read_csv("../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1")

# 1. Análisis de contenido de detalle de atención

In [7]:
df["detalle_atencion"][0]

'Paciente nueva primera consulta de Linares\nderivada del HLCM. \npaso a 3 medio (Quiere estudiar párvulos). \nCon diagnostico de  EP critica. Se intenta dilatación percutanea a los 3 ddv sin lograr apertura. Se reaiza cirugia en CEC comprobandose atresia pulmonar sin CIV y VD tripartito e insuficiencia tricuspidea moderada. Se realiza valvulotomia pulmonar mas plastia tricuspidea y plastia auricular el 30-9-1999(Dr. Leon). Evolucionando con Estenosis Residual moderada dilatacion de cavidades derechas y fx sistolica niormal. \nTrae RNM UC 2013:  VFD VD 108.6 ML/M2. IP FR 33.7% (Examen previo 20111: 15.4%). \nPRESENTO SINCOPE (Unico episodio) CON ESTUDIO NEGATIVO EN TIL TEST. \n\nEco hoy 12-1-16 Dr. Baraona: Estenosis pulmonar valvular y supravalvular leves con insuficiencia libre. Dilatación e hipertrofia moderada del VD con fx sistolica disminuida. Insuficiencia tricúspide moderada a severa. \n\nHace vida normal en gral, deporte al menos 4 veces por semana, ha bajado de peso y se sien